### Import Necessary Libraries

In [7]:
pip install -r requirements.txt

  Using cached streamlit-1.52.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached pandas-2.3.3-cp314-cp314-win_amd64.whl.metadata (19 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-win_amd64.whl.metadata (52 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached xgboost-3.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pillow-12.1.0-cp314-cp314-win_amd64.whl.metadata (9.0 kB)
  Using cached protobuf-6.33.2-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached pyarrow-22.0.0-cp314-cp314-win_amd64.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-a

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\visha\\AppData\\Local\\python\\pythoncore-3.14-64\\Lib\\site-packages\\fontTools\\ttLib\\tables\\_l_t_a_g.py'
Consider using the `--user` option or check the permissions.



In [8]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.0 MB 6.8 MB/s eta 0:00:02
   ------------- -------------------------- 3.1/9.0 MB 8.3 MB/s eta 0:00:01
   ----------------------- ---------------- 5.2/9.0 MB 9.2 MB/s eta 0:00:01
   --------------------------------- ------ 7.6/9.0 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 9.1 MB/s  0:00:01
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.1 MB 11.8 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.1 MB 11.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 11.2 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 10.3 MB/s  0:00:01
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   ----------------------

### Dataset Selection and Loading

**Requirements:**
- ≥500 samples
- ≥12 features
- Public dataset (UCI/Kaggle)
- Regression OR Classification problem

In [9]:
# Step 1: Load your dataset
# Dataset - https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

path = "data/data.csv"
print("Path to dataset files:", path)
data = pd.read_csv(path, delimiter=';')
print("Dataset loaded successfully.")

Path to dataset files: data/data.csv
Dataset loaded successfully.


### Data Pre-processing

In [10]:
# Step 2: Preprocess your data

# Step 2.1. Separate features (X) and target (y)
print(f"Columns in the loaded dataset: {list(data.columns)}\n")
print("Number of columns in the dataset:", len(data.columns))
print("Number of rows in the dataset:", len(data))

X = data.drop('Target', axis=1)

# Use mapping to create numeric target
y = data['Target']

print(f"First 5 entries of features data:")
display(X.head())
print(f"\nFirst 5 corresponding entries of target data:")
print(y[:5])

Columns in the loaded dataset: ['Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance\t', 'Previous qualification', 'Previous qualification (grade)', 'Nacionality', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Admission grade', 'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'International', 'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)', 'Unemployment rate', 'Inflation rate

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79



First 5 corresponding entries of target data:
0     Dropout
1    Graduate
2     Dropout
3    Graduate
4    Graduate
Name: Target, dtype: object


In [11]:
# Step 2.2 Handle missing values if any
print(f"Basic information about the dataset:")
data.info()
print(f"Number of missing values in the dataset: {data.isnull().sum().sum()}")
print(f"Number of duplicated rows in the dataset: {data.duplicated().sum()}")

Basic information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Fath

### Train-Test Split of Data

In [12]:
# Step 3: Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('✓ Dataset split into train, validation, and test sets successfully')

print("\nTrain set class distribution:")
print(pd.Series(y_train).value_counts())

print("\nTest set class distribution:")
print(pd.Series(y_test).value_counts())

✓ Dataset split into train, validation, and test sets successfully

Train set class distribution:
Target
Graduate    1767
Dropout     1137
Enrolled     635
Name: count, dtype: int64

Test set class distribution:
Target
Graduate    442
Dropout     284
Enrolled    159
Name: count, dtype: int64


In [13]:
# Save the test data with original labels for streamlit app use

test_data = X_test.copy()
test_data['Target'] = y_test

# Save to CSV
test_data.to_csv('data/test_data.csv', index=False)
print("✓ Test data saved to 'test_data.csv'")
print(f"\nFirst few rows of test data:")
display(test_data.head())

✓ Test data saved to 'test_data.csv'

First few rows of test data:


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1853,1,44,1,9003,1,39,160.0,1,3,3,...,0,6,7,6,14.666667,0,12.4,0.5,1.79,Graduate
2399,1,17,1,9500,1,1,131.0,1,1,1,...,0,8,10,8,11.712500,0,16.2,0.3,-0.92,Graduate
510,1,1,2,9070,1,1,121.0,1,4,1,...,0,6,9,0,0.000000,0,9.4,-0.8,-3.12,Enrolled
242,1,1,2,9147,1,1,118.0,1,37,19,...,0,5,5,5,13.600000,0,12.4,0.5,1.79,Graduate
3392,1,1,1,9070,1,1,133.1,1,1,1,...,0,6,6,6,12.666667,0,16.2,0.3,-0.92,Graduate


In [14]:
target_class_encoding = {
    cls: idx for idx, cls in enumerate(data['Target'].unique(), start=0)
}
# Save target class encoding mapping to CSV for use in deployment
mapping_df = pd.DataFrame(list(target_class_encoding.items()), columns=["class", "encoded"]) 
mapping_df.to_csv('model/target_class_encoding.csv', index=False)

y_train = y_train.map(target_class_encoding)
y_test = y_test.map(target_class_encoding)
print("✓ Target variable encoded successfully.")

✓ Target variable encoded successfully.


### Model Training & Evaluation

**Models to Implement:**
1. Logistic Regression
2. Decision Tree Classifier
3. K-Nearest Neighbor Classifier
4. Naive Bayes Classifier - Gaussian or Multinomial
5. Ensemble Model - Random Forest
6. Ensemble Model - XGBoost

In [16]:
# Step 4: Train various classification models and evaluate in training data

models = {
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42, criterion='entropy'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob, multi_class='ovr') if y_prob is not None else None,
        "Precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
        "F1": f1_score(y_test, y_pred, average='weighted', zero_division=0),
        "MCC": matthews_corrcoef(y_test, y_pred)
    })

    # Save the model
    with open(f"model/{name.replace(' ', '_')}.pkl", 'wb') as file:
        pickle.dump(model, file)

results_df = pd.DataFrame(results)
results_df

c:\Users\visha\AppData\Local\python\pythoncore-3.14-64\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 10000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=10000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy,AUC,Precision,Recall,F1,MCC
0,Logistic Regression,0.769492,0.875957,0.751411,0.769492,0.754174,0.616910
1,Decision Tree,0.649718,0.704741,0.656268,0.649718,0.652490,0.436606
2,KNN,0.593220,0.694784,0.564171,0.593220,0.569375,0.308273
3,Naive Bayes,0.659887,0.790761,0.634672,0.659887,0.643205,0.430308
4,Random Forest,0.766102,0.884911,0.752622,0.766102,0.751787,0.611211
5,XGBoost,0.762712,0.885778,0.755775,0.762712,0.757415,0.608789
